In [1]:
# Mounting drive to colab notebook
from  google.colab import drive
drive.mount('/content/drive/')
#change to current working directory
%cd 'drive/My Drive/nlp_project'
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/nlp_project
 bert					       'Clinical Segmentation ML.ipynb'
 bert_output				        data
 cased_L-12_H-768_A-12			        data_cleaned.tsv
'Clinical Note Segmentation using BERT.ipynb'   uncased_L-12_H-768_A-12


In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
from google.colab import files
uploaded = files.upload()

Saving data_cleaned.tsv to data_cleaned.tsv


In [4]:
df = pd.read_csv('data_cleaned.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 24: expected 4 fields, saw 10\nSkipping line 32: expected 4 fields, saw 80\nSkipping line 189: expected 4 fields, saw 6\nSkipping line 194: expected 4 fields, saw 6\nSkipping line 195: expected 4 fields, saw 6\nSkipping line 212: expected 4 fields, saw 5\nSkipping line 239: expected 4 fields, saw 6\nSkipping line 259: expected 4 fields, saw 44\nSkipping line 274: expected 4 fields, saw 6\nSkipping line 297: expected 4 fields, saw 22\nSkipping line 332: expected 4 fields, saw 6\nSkipping line 418: expected 4 fields, saw 22\nSkipping line 419: expected 4 fields, saw 60\nSkipping line 422: expected 4 fields, saw 36\nSkipping line 461: expected 4 fields, saw 10\nSkipping line 469: expected 4 fields, saw 758\nSkipping line 583: expected 4 fields, saw 36\nSkipping line 751: expected 4 fields, saw 6\nSkipping line 767: expected 4 fields, saw 6\nSkipping line 829: expected 4 fields, saw 10\nSkipping line 835: expected 4 fields, saw 22\nSkipping line 840: expected 4 fields, saw 

In [5]:
df.head()

,RECORD_ID,HEADER_TYPE,SECTION_TEXT,ORIGINAL_TEXT
0,299-05,social_history,\n;nothing new to report\n;nothing new to report,Social History:\n;nothing new to report\n;noth...
1,181-03,history_present_illness,"Briefly, Ms. Levin is a 54-year-old female ...","HISTORY OF PRESENT ILLNESS: Briefly, Ms. Levi..."
2,342-03,history_present_illness,\nMrs Barr has been doing well.\nHas a 8 weeks...,History of Present Illness\nMrs Barr has been ...
3,342-03,medications,\nPlavix (CLOPIDOGREL) 75 MG (75MG TABLET take...,Medications\nPlavix (CLOPIDOGREL) 75 MG (75MG ...
4,343-02,history_present_illness,HISTORY OF PRESENTING COMPLAINT Mr. Killian ...,HISTORY OF PRESENTING COMPLAINT: Mr. Killian ...


In [6]:
df['SECTION_TEXT'][1]

'   Briefly, Ms. Levin is a 54-year-old female with diabetes, hypertension, hypercholesterolemia, who has known coronary artery disease.\\nShe had an exercise test in 2076 for some vague symptoms, which was positive, and led to cardiac catheterization.\\nThis catheterization revealed 60% LAD lesion, 50% first marginal, 60% RCA, and a 70% diagonal lesion.\\nShe was maintained on medical management with few symptoms.\\nShe has been lost to followup from Cardiology Clinic.\\nShe has within the last week developed increasing symptoms with minimal exertion.\\nShe underwent a stress test in which she was able to exercise for 6 minutes and 9 seconds on the Bruce protocol.\\nShe achieved 93% of age-predicted maximal heart rate.\\nAt 4 minutes, she developed 2 mm ST segment depressions in the anterior leads associated with chest discomfort.\\nShe is a vague historian, but has not had clear rest anginal symptoms.\\nShe had an exercise test in 2076 for some vague symptoms, which was positive, and

In [7]:
df.shape

(1933, 4)

In [8]:
df = df.dropna()
df.shape

(1923, 4)

In [9]:
df['HEADER_TYPE'].value_counts()

history_present_illness    789
medications                662
social_history             472
Name: HEADER_TYPE, dtype: int64

In [10]:
df.head()

,RECORD_ID,HEADER_TYPE,SECTION_TEXT,ORIGINAL_TEXT
0,299-05,social_history,\n;nothing new to report\n;nothing new to report,Social History:\n;nothing new to report\n;noth...
1,181-03,history_present_illness,"Briefly, Ms. Levin is a 54-year-old female ...","HISTORY OF PRESENT ILLNESS: Briefly, Ms. Levi..."
2,342-03,history_present_illness,\nMrs Barr has been doing well.\nHas a 8 weeks...,History of Present Illness\nMrs Barr has been ...
3,342-03,medications,\nPlavix (CLOPIDOGREL) 75 MG (75MG TABLET take...,Medications\nPlavix (CLOPIDOGREL) 75 MG (75MG ...
4,343-02,history_present_illness,HISTORY OF PRESENTING COMPLAINT Mr. Killian ...,HISTORY OF PRESENTING COMPLAINT: Mr. Killian ...


In [0]:
from gensim.models import doc2vec
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the clinical notes.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [0]:

X_train, X_test, y_train, y_test = train_test_split(df.SECTION_TEXT, df.HEADER_TYPE, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [13]:
len(all_data)

1923

In [14]:
all_data[:1]

[TaggedDocument(words=['history;', 'no', 'focal', 'neurologic', 'signs.'], tags=['Train_0'])]

In [15]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 1923/1923 [00:00<00:00, 343701.65it/s]


In [16]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1923/1923 [00:00<00:00, 1862305.84it/s]


CPU times: user 1min 17s, sys: 468 ms, total: 1min 18s
Wall time: 41.3 s


In [0]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [0]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [19]:
train_vectors_dbow.shape,y_train.shape

((1346, 300), (1346,))

In [20]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(multi_class='multinomial', solver = 'lbfgs')
logreg.fit(train_vectors_dbow, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
y_pred = logreg.predict(test_vectors_dbow)
np.mean(y_pred == y_test)

0.9081455805892548

In [24]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(train_vectors_dbow, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
y_pred_NB = model.predict(test_vectors_dbow)

In [27]:
np.mean(y_pred_NB == y_test)

0.8370883882149047

In [28]:
logreg.score(test_vectors_dbow, y_test)

0.9081455805892548

In [29]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import f1_score
print("f1_score (micro) = " + str(f1_score(y_test, y_pred, average = 'micro')))
print("f1_score (weighted) = " + str(f1_score(y_test, y_pred, average = 'weighted')))
print("f1_score (macro) = " + str(f1_score(y_test, y_pred, average = 'macro')))

[[219   7  17]
 [  5 176   6]
 [  8  10 129]]
f1_score (micro) = 0.9081455805892548
f1_score (weighted) = 0.9083803135935805
f1_score (macro) = 0.9037657689373937
